<a href="https://colab.research.google.com/github/whaldsz/deep-learning/blob/main/Disease_Indicators_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Disease Prediction



## Setup and initialization

In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import math

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from keras.utils.vis_utils import plot_model
import os

#!pip install -q git+https://github.com/tensorflow/docs 


import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

print("TensorFlow version::", tf.__version__)


from sklearn.model_selection import train_test_split

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [ ]:
from  IPython import display
from matplotlib import pyplot as plt

import numpy as np

import pathlib
import shutil
import tempfile

In [ ]:
# currentdir
logdir = os.path.join(os.getcwd(), "tensorboard_logs")
shutil.rmtree(logdir, ignore_errors=True)

In [ ]:
# set random seed

tf.random.set_seed(74)

## 1. Dataset Preparation

In [ ]:
disease_training = pd.read_csv('C:/Users/whald/Google Drive/projects/oman-gulf-college/dataset/Disease_Prediction/Training.csv')
disease_testing = pd.read_csv('C:/Users/whald/Google Drive/projects/oman-gulf-college/dataset/Disease_Prediction/Testing.csv')
disease_training.head()

#3 Remove last column

In [ ]:
disease_training.isna().sum()

In [ ]:
disease_training.drop('Unnamed: 133', inplace=True, axis=1)

disease_training.isna().sum()

In [ ]:
#disease_training.head()

## Convert category to numeric values

In [ ]:
#get class labels

class_names = np.unique(disease_training.prognosis)
disease_training.prognosis = pd.Categorical(disease_training.prognosis)
disease_testing.prognosis = pd.Categorical(disease_testing.prognosis)
class_names.shape

In [ ]:
#disease_training.prognosis.cat.codes
#disease_training
#disease_testing.head()

## Separate Features and Label - Training

### Training Set

In [ ]:
X = disease_training.drop('prognosis', axis=1)
y = disease_training.prognosis.cat.codes
np.unique(y)

### Unseen Test Set

In [ ]:
X_unseen = disease_testing.drop('prognosis', axis=1)
y_unseen = disease_testing.prognosis.cat.codes
np.unique(X_unseen)

## Split into Training & Validation Test

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=74)
#print(X_train.shape)
#print(y_train.shape)
#print(X_test.shape)
#print(y_test.shape)


NameError: name 'train_test_split' is not defined

In [2]:
X_test

NameError: name 'X_test' is not defined

In [ ]:
# Number of features
FEATURES = X_test.shape[1]
FEATURES

## 2. Model Training

### Training configuration

In [ ]:
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(1e4)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

### Create Model

### Find the ideal learning rate

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

In [ ]:
step = np.linspace(0,100000)
lr = lr_schedule(step)
plt.figure(figsize = (6,4))
plt.plot(step/STEPS_PER_EPOCH, lr)
plt.ylim([0,max(plt.ylim())])
plt.xlabel('Epoch')
_ = plt.ylabel('Learning Rate')

In [ ]:
#lrs = 1e-4 * (10 ** (tf.range(40)/20))
#plt.figure(figsize=(6,4))
#plt.semilogx(lrs, history.history['loss'])
#plt.xlabel("Learning Rate")
#plt.ylabel("Loss")
#plt.title("Learning Rate vs Loss")

### Settings for automation

In [ ]:
def get_callbacks(name):
  return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=200),
    tf.keras.callbacks.TensorBoard(os.path.join(logdir,name)),
  ]

In [ ]:
def compile_and_fit(model, name, loss=None, optimizer=None, metrics = None, max_epochs=10000):
  if optimizer is None:
    optimizer = get_optimizer()

  if loss is None:
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  if metrics is None:
    metrics = ['accuracy']
  model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=metrics
  )

  model.summary()

  history = model.fit(
    X_train,
    y_train,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs=max_epochs,
    validation_split=0.1,
    #validation_data=[X_test, y_test],
    callbacks=get_callbacks(name),
    verbose=0)
  return history

### Models

In [ ]:
size_histories = {}

#### Model 1

Simple model with 3 layers

In [ ]:
model_1 = tf.keras.Sequential([
    layers.Dense(4, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(41)
])

In [ ]:
size_histories['model1'] = compile_and_fit(
    model_1, 
    'models/model1', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy()
)

#### Model 2

In [ ]:


model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(4, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(41, activation=tf.keras.activations.softmax)
])

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                #metrics=['MultiClassConfusionMatrixPlot'])
                metrics=["accuracy"])

scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10 **(epoch/20))

history = model.fit(X_train, y_train, epochs=40, callbacks=[scheduler])

#### Model 3

#### Model 4

#### Model 5

### 3. Evaluate Model

#### Evaluate with test data

In [ ]:
loss, acc = model_1.evaluate(X_test, y_test)
print(f"Model Loss (Test Set) : {loss}")
print(f"Model Accuracy (Test Set): {acc}")

#### Evaluate with unseen data

In [ ]:
loss, acc = model.evaluate(X_unseen, y_unseen)
print(f"Model Loss (Unseen Data Set) : {loss}")
print(f"Model Accuracy (Unseen Data Set): {acc}")

## 3. Testing

#### Test Set

In [ ]:
### Test set
predictions = model.predict(X_test)
predictions[0]

In [ ]:
result = class_names[tf.argmax(predictions, axis=1)]


#### Unseen Test Data

In [ ]:
prediction_unseen_data = model.predict(X_unseen)
prediction_unseen_data

In [ ]:
result = class_names[tf.argmax(prediction_unseen_data, axis=1)]
result

### Correlation Matrix

In [ ]:
#plt.figure(figsize=(25,12))

#sns.heatmap(disease_training.corr().abs(), annot=True)
